In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install contractions
!pip install pandas
import nltk
nltk.download('stopwords')

! cp 'drive/MyDrive/Bachelor Project/Constants.py' .
! cp 'drive/MyDrive/Bachelor Project/TweetsProcessingToolkits.py' .


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from TweetsProcessingToolkits import min_distance, process_tweet
from Constants import data_columns, diseases_dict, data_path, dates, common_cancer_types
import pandas as pd
import json as js

## Load data

* As all the data is srcaped and saved by month, restore them all together for each disease

In [4]:
#  first gather all raw data into one csv file
file_names = {disease: [(disease + '_'+ date + '.csv') for date in dates] for disease in diseases_dict.keys()}
#print(file_names)

for disease in file_names.keys():
  tweets_list = list()
  for fname in file_names[disease]:
    #fname.replace('')
    data = pd.read_csv(data_path + 'raw_data/'+ fname.replace(':','_'))
    data_count = data.shape[0]
    for i in range(data_count):
      tweets_list.append([data[column][i] for column in data_columns])

  tweets_df = pd.DataFrame(tweets_list, columns=data_columns)
  tweets_df.to_csv(data_path + disease + '.csv', index=False)
  print(f'{disease} files saved data successfully')

cancer files saved data successfully
hypertension files saved data successfully
hiv files saved data successfully
hyperlipidemia files saved data successfully
diabetes files saved data successfully
depression files saved data successfully
asthma files saved data successfully
arthritis files saved data successfully
anxiety files saved data successfully


In [5]:
!pip install snscrape
!pip install pandas
!pip3 install snscrape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 3.1 MB/s eta 0:00:00


* load data into a dictionary by the below form


```
data_dict = {'cancer':{'datetime': [], 'id': [], 'text': [], 'username': []}
            'hypertension': {}...
            }
```



In [6]:
data_dict = dict()
for disease in diseases_dict:
    data = pd.read_csv(data_path + disease + '.csv')

    data_dict.update({disease: dict()})

    data_count = data.shape[0]
    for column in data_columns:
        data_dict[disease].update({column: [data[column][i] for i in range(data_count)]})
    print('{}: {}'.format(disease, data_count))

cancer: 58015
hypertension: 1718
hiv: 2596
hyperlipidemia: 235
diabetes: 8840
depression: 26947
asthma: 2777
arthritis: 2770
anxiety: 27042


## Evaluate scraped data


### **Step 1:**
**Separate those tweets that includes each disease name completely in the tweet's text**

The reason that we only consider those include _"diagnosed with"_ not _"diagnosed"_ is that the verb diagnosed without with, is used when so. diagnose sth. therefore it's used by Dr., not patients

* devide the data into seperate dictonary based on how the disease name has been mentioned in the tweet text (eg: aids, hiv)


```
data_dict = {'cancer':
          {'cancer': {'datetime': [], 'id': [], 'text': [], 'username': [], 'normalized_text' : []}
'hypertension': {'hypertension':{},
			'hypertention', ...}, …
}

```

In [7]:
data_subsection_dict = {
    disease: {disease_keyword: {column: [] for column in (data_columns + ['normalized_text']) } for disease_keyword in diseases_dict[disease]} for disease in
    diseases_dict}

for disease in data_dict:
    disease_keywords = diseases_dict[disease]

    # add a column that includes tweets that the tweet text has none of the disease name but was scraped as one! (just for statistical matter)
    data_subsection_dict[disease].update({'none': {column: [] for column in (data_columns + ['normalized_text'])}})

    for index, tweet in enumerate(data_dict[disease]['text']):

        none = True
        # remove links and punctuation before searching for the disease name in the tweet text and convert all chars into lower case
        tweet = process_tweet(tweet).lower()

        for keyword in disease_keywords:


            # as high is an adjective, it can appears in different order, so do not consider it like: high blood presure and blood pressure is high
            if 'high'  in keyword:
                search_keyword = keyword.replace('high ', '')
                # but the 'high' word have to be found in the tweet text
                if search_keyword in tweet and 'high' in tweet:

                    for column in data_columns:
                      data_subsection_dict[disease][keyword][column].append(data_dict[disease][column][index])

                    # add normalized tweet text for further use
                    data_subsection_dict[disease][keyword]['normalized_text'].append(tweet)

                    # found a disease name in tweet, turn the none variable to False
                    none = False
            else: # there is not 'high' in the disease name
                if keyword in tweet:

                    for column in data_columns:
                      data_subsection_dict[disease][keyword][column].append(data_dict[disease][column][index])

                    data_subsection_dict[disease][keyword]['normalized_text'].append(tweet)

                    # found a disease name in tweet, turn the none variable to False
                    none = False


        if none: # it shows that non of the diseases name have been found in the tweet text
            for column in data_columns:
              data_subsection_dict[disease]['none'][column].append(data_dict[disease][column][index])

            data_subsection_dict[disease]['none']['normalized_text'].append(tweet)


**See which aount of each disease's data has been eliminate**

(scraped as they included the disease name in twitter search, but the disease name has not been found in the tweet text)

In [8]:
for disease in data_subsection_dict:
    print('{:20}'.format(disease))
    for subsection in data_subsection_dict[disease]:
        print('\t{:30}: {}'.format(subsection, len(data_subsection_dict[disease][subsection]['text'])))

cancer              
	cancer                        : 58007
	none                          : 8
hypertension        
	hypertension                  : 833
	hypertention                  : 2
	high blood pressure           : 807
	high bp                       : 225
	none                          : 0
hiv                 
	hiv                           : 1919
	aids                          : 1243
	none                          : 0
hyperlipidemia      
	hyperlipidemia                : 12
	lipid disorder                : 0
	 hypercholesterolemia         : 9
	high blood cholesterol        : 0
	high cholesterol              : 216
	none                          : 0
diabetes            
	diabetes                      : 8830
	dt1                           : 6
	dt2                           : 6
	none                          : 7
depression          
	depression                    : 26945
	none                          : 2
asthma              
	asthma                        : 2777
	none              

In [9]:
for disease in data_subsection_dict:
    print('{:20}'.format(disease))
    for subsection in data_subsection_dict[disease]:
      index = int(len(data_subsection_dict[disease][subsection]['text'])/2)
      try:
        print('\t{:30}: {}'.format(subsection, data_subsection_dict[disease][subsection]['text'][index]))
      except:
        pass

cancer              
	cancer                        : @KirstyWebeck I have two friends diagnosed with breast cancer over the last few months. Do not put off getting checked out. 🙏
	none                          : It all started so simply.  One day he just couldn't jump up on my lap.  Kinda shuffled back and forth with frustration.

Two days later he had an odd limp.

The next day he was at the vet and diagnosed with arthritis.  It's weird thinking back to how upset I was to hear that.
hypertension        
	hypertension                  : Yep! I was diagnosed with Gestational Hypertension from the stress of birthing my child. Didn’t have high blood pressure my whole pregnancy except for during my labor. https://t.co/FLkxv6Ggjm
	hypertention                  : Diagnosed with Essential Hypertention. 😪

My Blood Pressure elevated after I got vaccinated as I saw the trend moving up for quite sometime now. The doctor also told me that it is possible (We are not blaming the vaccine itself). T

### **Step 2:**
**Check the distance between the terms _"I"_ and _"Diagnosed"_ in each tweet, afterwards only consider those with the distances 1 or 2 (which is filled with verbs like was, am, ...)**

* find the mentioned distance for each tweet

In [10]:
distances_dict = {disease: { subsection:[] for subsection in data_subsection_dict[disease].keys()} for disease in data_subsection_dict.keys()}

for disease in data_subsection_dict:
    for subsection in data_subsection_dict[disease]:
        for tweet in data_subsection_dict[disease][subsection]['normalized_text']:
            dist = min_distance(tweet, 'i', 'diagnosed')
            if not dist: # the text either does not includes one of the words or they are not apeared in order (first 'i' and then 'diagnosed')
                distances_dict[disease][subsection].append(-1)
            else:
                distances_dict[disease][subsection].append(dist)


consider those with the distance greater than zero and less than three

In [11]:
first_filtered_data_dict = {disease: {subsection: {column: list() for column in (data_columns+ ['normalized_text'])} for subsection in (list(data_subsection_dict[disease]) + ['first_filtered'])} for disease in data_subsection_dict}

for disease in distances_dict:
  for subsection in distances_dict[disease]:
    for index, dist in enumerate(distances_dict[disease][subsection]):
      if dist < 3 and dist > 0:
        for column in (data_columns+ ['normalized_text']):
            first_filtered_data_dict[disease][subsection][column].append(data_subsection_dict[disease][subsection][column][index])
      else: # i and diagnosed either didnt appeared in order or they were in a distance greater than 2
        for column in (data_columns+ ['normalized_text']):
          first_filtered_data_dict[disease]['first_filtered'][column].append(data_subsection_dict[disease][subsection][column][index])

In [12]:
for disease in first_filtered_data_dict:
  print(disease)
  for subsection in first_filtered_data_dict[disease]:
    if subsection != 'none' and subsection != 'first_filtered':
      print('{:10} {} : {}'.format('',subsection, len(first_filtered_data_dict[disease][subsection]['id'])))

cancer
           cancer : 15780
hypertension
           hypertension : 421
           hypertention : 0
           high blood pressure : 390
           high bp : 103
hiv
           hiv : 695
           aids : 381
hyperlipidemia
           hyperlipidemia : 8
           lipid disorder : 0
            hypercholesterolemia : 3
           high blood cholesterol : 0
           high cholesterol : 118
diabetes
           diabetes : 3890
           dt1 : 0
           dt2 : 1
depression
           depression : 13943
asthma
           asthma : 1135
arthritis
           arthritis : 1305
anxiety
           anxiety : 13348


In [13]:
for disease in first_filtered_data_dict:
  print(disease)
  for subsection in first_filtered_data_dict[disease]:
    if subsection != 'none' and subsection != 'first_filtered':
      index = int(len(first_filtered_data_dict[disease][subsection]['id']) /2)
      try:
        print('{:10} {} : {}'.format('',subsection, first_filtered_data_dict[disease][subsection]['text'][index]))
      except:
        pass

cancer
           cancer : Furthermore when I got diagnosed with cancer, we were relieve we did not have to make choices between my life and that embryo’s life.
hypertension
           hypertension : Small life update!

After suffering from persistent headaches for the last 3 weeks, I was taken to hospital late Monday night. Turns out my blood pressure was very high - a "you're in danger of having a stroke" high, and it's possible I may be diagnosed with Hypertension.

/1
           high blood pressure : Do statistics are real. I was more likely to get a heart disease than find a love of my life (or at least the first one). And tramparara I was diagnosed with a high blood pressure. Life os good and getting shorter each day, have fun.
           high bp : @TheCuradhan @KySportsRadio I was diagnosed with high blood pressure after being exposed to someone with COVID-19. My BP was 190/115 when I went to the hospital in March 2020. 
It took months to get it under control, but I’m doing much

### **Step 3:**
**Check the distance between the terms "diagnosed" and "< disease name >" , afterwards only consider those with distances 1**

In [15]:
disease_distances_dict = {disease: {subsection: [] for subsection in first_filtered_data_dict[disease]} for disease in first_filtered_data_dict}

for disease in first_filtered_data_dict:
    for subsection in first_filtered_data_dict[disease]:
      if subsection != 'none' and subsection != 'first_filtered': # these sectiones have been eliminated previously
        if 'high' in subsection:
            subsection_wo_high = subsection.replace('high ', '')

            for tweet in first_filtered_data_dict[disease][subsection]['normalized_text']:
                # distance = -1
                dist_1 = min_distance(tweet, 'diagnosed', subsection)
                dist_2 = min_distance(tweet,'diagnosed', subsection_wo_high)

                if not dist_1 and not dist_2: # there is no sequence of "diagnosed" and "<first term of the disease name>"
                    distance = -1
                elif not dist_1: # there is a sequence like diagnosed ... <disease name w/o high at the first>
                    # only add it if the distance between "high" and rest of the name is equal to 1 (like: blood pressure is high)
                    high_dist = min_distance(tweet,subsection_wo_high ,'high')
                    if high_dist == 1:
                        distance = dist_2
                else: # there is a sequence like diagnosed ... <disease name w/ high at first>
                    distance = dist_1

                disease_distances_dict[disease][subsection].append(distance)
        elif subsection == 'cancer':
          # check if the distance between diagnosed and cancer is more than one due to including the cancer name, then include this tweet in dataset
          for tweet in first_filtered_data_dict[disease][subsection]['normalized_text']:
            dist_1 = min_distance(tweet, 'diagnosed', subsection)
            dist_2 = False
            for cancer_type in common_cancer_types:
              dist = min_distance(tweet, 'dignosed', cancer_type + ' ' + subsection)
              if not dist:
                continue
              else:
                if (not dist_2)  or dist < dist_2:
                  dist_2 = dist
            if not dist_1 and not dist_2:
              #there is no sequence like diagnosed ... cancer and diagnosed ... <cancer_type> cancer
              disease_distances_dict[disease][subsection].append(-1)
            elif not dist_2:
              # there is a sequence like diagnosed .. cancer but not any like diagnosed ... <cancer_type> cancer
              disease_distances_dict[disease][subsection].append(dist_1)
            else:
              # there is a sequence like diagnosed ... cancer and another like diagnosed ... <cancer_type> cancer,
              # in this case, we add the min of these as a distance between diagnosed and disease name
              dist = min(dist_1, dist_2)
              disease_distances_dict[disease][subsection].append(dist)

        else:  # disease name does not include "high" at first
            for tweet in first_filtered_data_dict[disease][subsection]['normalized_text']:
                # only consider first term of the disease name
                dist = min_distance(tweet, 'diagnosed', subsection)
                if not dist:    # there is no sequence of "diagnosed" and "<first term of the disease name>"
                    disease_distances_dict[disease][subsection].append(-1)
                else:           # there is a sequence like diagnosed ... <disease name>
                    disease_distances_dict[disease][subsection].append(dist)



consider those with the distance equals to 1

In [16]:
second_filtered_data_dict = {disease: {subsection: {column: list() for column in (data_columns+ ['normalized_text'])} for subsection in (list(first_filtered_data_dict[disease]) + ['second_filtered'])} for disease in first_filtered_data_dict}

for disease in disease_distances_dict:
  for subsection in disease_distances_dict[disease]:
    if subsection == 'none' or subsection == 'fist_filtered':
      for column in second_filtered_data_dict[disease][subsection]:
        second_filtered_data_dict[disease][subsection] = first_filtered_data_dict[disease][subsection]
    else:
      for index, dist in enumerate(disease_distances_dict[disease][subsection]):
        if dist == 1:
          for column in second_filtered_data_dict[disease][subsection]:
            second_filtered_data_dict[disease][subsection][column].append(first_filtered_data_dict[disease][subsection][column][index])
        else:
          for column in second_filtered_data_dict[disease]['second_filtered']:
            second_filtered_data_dict[disease]['second_filtered'][column].append(first_filtered_data_dict[disease][subsection][column][index])


In [17]:
for disease in second_filtered_data_dict:
  print(disease)
  for subsection in second_filtered_data_dict[disease]:
    if subsection not in ('none','first_filtered','second_filtered'):
      print('{:10} {} : {}'.format('',subsection, len(second_filtered_data_dict[disease][subsection]['id'])))

cancer
           cancer : 4204
hypertension
           hypertension : 156
           hypertention : 0
           high blood pressure : 191
           high bp : 42
hiv
           hiv : 324
           aids : 79
hyperlipidemia
           hyperlipidemia : 0
           lipid disorder : 0
            hypercholesterolemia : 0
           high blood cholesterol : 0
           high cholesterol : 43
diabetes
           diabetes : 1358
           dt1 : 0
           dt2 : 1
depression
           depression : 4718
asthma
           asthma : 556
arthritis
           arthritis : 286
anxiety
           anxiety : 2573


In [18]:
for disease in second_filtered_data_dict:
  print(disease)
  for subsection in second_filtered_data_dict[disease]:
    if subsection not in ('none','first_filtered','second_filtered'):
      # index = int(len(second_filtered_data_dict[disease][subsection]['id'])/4)
      try:
        print('{:10} {} : {}'.format('',subsection, second_filtered_data_dict[disease][subsection]['normalized_text'][16]))
      except:
        pass

cancer
           cancer : @ analysingcrypto @ cz _ binance @ markyusko @ coindesk please help i have been diagnosed with cancer . we need funds for treatment , while the initial stage . save me btc : 344yevyrrhgj6e8ys99mmqxkorgq3mjopn or a retweet . thank you to all the good people
hypertension
           hypertension : @ mhlangatpn the meeting pic is a joke part of my tweet . . . anyway , i have been told many times to take my healthy serious since i was diagnosed with hypertension in 2010 and it is only in 2017 that i began to walk for exercise . now , i am doing very well with running in my exercises . . .
           high blood pressure : i was 14 diagnosed with high blood pressure because i was fat . the doctors did not run tests o . meanwhile i just had sinusitis . they put me on hbd medication . at 14 ! ! ! ! my mother had to fight them and took me to a different doctor that listened .
           high bp : @ bennedose @ mohansinha i got diagnosed with high bp at 32 and i was not

In [19]:
with open(data_path + 'final_data.json', 'w') as final_file:
  js_serializable_final_dict = second_filtered_data_dict.copy()
  for disease in js_serializable_final_dict:
    for subsection in js_serializable_final_dict[disease]:
      js_serializable_final_dict[disease][subsection]['id'] = [int(datetime) for datetime in second_filtered_data_dict[disease][subsection]['id']]
      js_serializable_final_dict[disease][subsection]['datetime'] = [str(datetime) for datetime in second_filtered_data_dict[disease][subsection]['datetime']]
  js.dump(js_serializable_final_dict, final_file)

## Save users with their tweets date



In [20]:
users_dict = {disease: {disease_keyword:dict() for disease_keyword in diseases_dict[disease]} for disease in
    diseases_dict}

for disease in users_dict:
  for subsection in users_dict[disease]:
    for username, datetime  in zip(second_filtered_data_dict[disease][subsection]['username'], second_filtered_data_dict[disease][subsection]['datetime']):
      try:
        users_dict[disease][subsection][username].append(datetime)
      except KeyError:
        users_dict[disease][subsection].update({username: [datetime]})


In [21]:
#Number of users

unique_users = set()

# Iterate over diseases in users_dict
for disease in users_dict:
    # Iterate over subsections in users_dict[disease]
    for subsection in users_dict[disease]:
        # Add usernames to the set
        unique_users.update(users_dict[disease][subsection].keys())

# Get the number of unique users
num_users = len(unique_users)

# Print the result
print("Number of users:", num_users)


Number of users: 12783


In [22]:
with open(data_path + 'users.json', 'r') as users_file:
  users_dict = js.load(users_file)
  for disease in users_dict:
    print(disease)
    for subsection in users_dict[disease]:
      print('{:10s} {}: {}'.format('',subsection, len(users_dict[disease][subsection])))

cancer
           cancer: 3328
hypertension
           hypertension: 132
           hypertention: 0
           high blood pressure: 164
           high bp: 38
hiv
           hiv: 260
           aids: 59
hyperlipidemia
           hyperlipidemia: 0
           lipid disorder: 0
            hypercholesterolemia: 0
           high blood cholesterol: 0
           high cholesterol: 38
diabetes
           diabetes: 1235
           dt1: 0
           dt2: 1
depression
           depression: 4499
asthma
           asthma: 534
arthritis
           arthritis: 271
anxiety
           anxiety: 2453


In [23]:
with open(data_path + 'users.json', 'w') as users_file:
  js.dump(users_dict, users_file)

## scrape users data

Install the proper python inorder to run .py file and scare users data


In [ ]:
!apt-get install python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3.8 get-pip.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 4,693 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.8-minimal amd64 3.8.15-1+bionic1 [762 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.8-minimal amd64 3.8.15-1+bionic1 [1,838 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 li

In [ ]:
!pip --version

pip 22.3.1 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)


run a .py code that scrape each users data in the given time spam

In [ ]:
!python3.8 'drive/MyDrive/Bachelor Project/UserSearch.py'

Traceback (most recent call last):
  File "drive/MyDrive/Bachelor Project/UserSearch.py", line 1, in <module>
    from Constants import diseases_dict, data_path, data_columns
  File "/content/drive/MyDrive/Bachelor Project/Constants.py", line 4, in <module>
    from nltk.corpus import stopwords
ModuleNotFoundError: No module named 'nltk'


## Restore all collected data in a sigle file


* for each chosen disease, users' data have been stored in a separate file for each alternative names of chosen disease, sum them up in a dictionary; then store all diseases' users's data in a file named 'collected_users_data'

In [ ]:
users_data_dict = {disease: dict() for disease in diseases_dict}
failed_users_data = {disease:{subsection: dict() for subsection in diseases_dict[disease]} for disease in diseases_dict}
for disease in diseases_dict: # for each disease
    for subsection in diseases_dict[disease]: # for each alternative name for the chosen disease
      # file path to the file that store users data for the disease and the alterantive used name
      file_name = data_path + 'Users/'+ '{}_{}_users_data.json'.format(disease,subsection)

      try:  # try to open the file

        with open(file_name, 'r') as users_file: # load the data into memory
          sub_users_dict = js.load(users_file)

          for username in sub_users_dict: # for each user in this file

            if len(sub_users_dict[username]) > 1: # those users that were scraped
              users_data_dict[disease].update({username:sub_users_dict[username]})
            else: # those users that an error accured while scraping
              failed_users_data[disease][subsection].update({username: sub_users_dict[username]})

      except FileNotFoundError: # this file does no exist
        print(f'Error: there is no file with "{file_name}" address')


# save all users data into a single file
with open(data_path + 'collected_users_data.json', 'w') as users_data_file:
  js.dump(users_data_dict, users_data_file)

# store those users who has not had any tweet
with open(data_path + 'failed_users_data.json', 'w') as users_data_file:
  js.dump(users_data_dict, users_data_file)



* an overview of users count for each disease

In [ ]:

for disease in diseases_dict:
  print(disease)
  found = len(users_data_dict[disease])
  failed = sum([len(failed_users_data[disease][subsection]) for subsection in diseases_dict[disease]])
  print('{:10}found: {}\n{:10}failed: {}'.format('', found, '', failed))

cancer
          found: 2645
          failed: 0
hypertension
          found: 326
          failed: 0
hiv
          found: 259
          failed: 0
hyperlipidemia
          found: 38
          failed: 0
diabetes
          found: 1235
          failed: 0
depression
          found: 4326
          failed: 0
asthma
          found: 534
          failed: 0
arthritis
          found: 271
          failed: 0
anxiety
          found: 977
          failed: 0


* Users who at least have tweeted once before and once after they've mentioned being diagnosed with the chosen disease

In [ ]:
for disease in diseases_dict:
  count =0

  for username in users_data_dict[disease]:
    if len(users_data_dict[disease][username]['before']['id']) >0 and len(users_data_dict[disease][username]['after']['id']) > 0 :
      count +=1
  print(disease, ':', count)

cancer : 2514
hypertension : 312
hiv : 240
hyperlipidemia : 34
diabetes : 1182
depression : 3963
asthma : 506
arthritis : 260
anxiety : 905


* USers who at least have tweeted once before they've mentioned being diagnosed with the chosen disease

In [ ]:
for disease in diseases_dict:
  count =0

  for username in users_data_dict[disease]:
    if len(users_data_dict[disease][username]['before']['id']) > 0 :
      count +=1
  print(disease, ':', count)

cancer : 2563
hypertension : 317
hiv : 244
hyperlipidemia : 36
diabetes : 1189
depression : 3995
asthma : 510
arthritis : 261
anxiety : 911
